<a href="https://colab.research.google.com/github/slightperturbation/ml_examples/blob/master/ML_Examples_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine Tuning Inception

In [1]:
import os
import tensorflow as tf

In [16]:
# Download the model and the weights
inception = tf.keras.applications.inception_v3.InceptionV3(input_shape = (75, 75, 3),
                        include_top = False)

# Lock it so it doesn't update during backpropagation
for layer in inception.layers:
  layer.trainable = False

# Get a handle to the last layer to use before our added fully connected layers.
last_inception_layer = inception.get_layer('mixed7')
#inception.summary()

Add our own trainable layers on top of the pre-trained CNNs.

In [22]:
# Final layers to learn the classifier
x = tf.keras.layers.Flatten()(last_inception_layer.output)
x = tf.keras.layers.Dense(1024, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dropout(0.2)(x)                  
x = tf.keras.layers.Dense(10, activation=tf.nn.softmax)(x)

model_base = tf.keras.Model(inception.input, x)

# Add a new layer at the beginning to resize the image up to Inception size.
input = tf.keras.Input(shape=(32,32,3))
preprocess = tf.keras.layers.experimental.preprocessing.Resizing(75, 75)(input)
preprocess = model_base(preprocess)

model = tf.keras.Model(input, preprocess)

model.summary()

Model: "functional_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
resizing_11 (Resizing)       (None, 75, 75, 3)         0         
_________________________________________________________________
functional_16 (Functional)   (None, 10)                16064426  
Total params: 16,064,426
Trainable params: 7,089,162
Non-trainable params: 8,975,264
_________________________________________________________________


In [19]:
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics = ['accuracy'])

In [20]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0


170500096/170498071 [==============================] - 6s 0us/step


In [21]:
history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

Epoch 1/10
1563/1563 [==============================] - 26s 17ms/step - loss: 1.7588 - accuracy: 0.7117 - val_loss: 1.6988 - val_accuracy: 0.7652
Epoch 2/10
1563/1563 [==============================] - 25s 16ms/step - loss: 1.6813 - accuracy: 0.7836 - val_loss: 1.6869 - val_accuracy: 0.7762
Epoch 3/10
1563/1563 [==============================] - 25s 16ms/step - loss: 1.6536 - accuracy: 0.8105 - val_loss: 1.6815 - val_accuracy: 0.7806
Epoch 4/10
1563/1563 [==============================] - ETA: 0s - loss: 1.6350 - accuracy: 0.8288

KeyboardInterrupt: ignored